In [2]:
pip install yt-dlp openai-whisper transformers torch torchaudio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/

In [3]:
import yt_dlp
import whisper
import torch
from transformers import pipeline
import os

# Define file paths
VIDEO_FILE = "video.mp4"
AUDIO_FILE = "audio.mp3"

# Step 1: Download YouTube Video
def download_video(youtube_url, output_path=VIDEO_FILE):
    ydl_opts = {
        'outtmpl': output_path,
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'merge_output_format': 'mp4',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

# Step 2: Extract Audio from Video
def extract_audio(video_path=VIDEO_FILE, audio_path=AUDIO_FILE):
    os.system(f"ffmpeg -i {video_path} -q:a 0 -map a {audio_path} -y")

# Step 3: Transcribe Audio using Whisper
def transcribe_audio(audio_path=AUDIO_FILE):
    model = whisper.load_model("small")  # Load Whisper model
    result = model.transcribe(audio_path)
    return result["text"]

# Step 4: Summarize Text using BART
def summarize_text(text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)
    summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
    return summary[0]['summary_text']

# Main function to run the entire pipeline
def video_summarization_pipeline(youtube_url):
    print("[1] Downloading Video...")
    download_video(youtube_url)

    print("[2] Extracting Audio...")
    extract_audio()

    print("[3] Transcribing Audio...")
    extracted_text = transcribe_audio()
    print("\nExtracted Text Preview:", extracted_text[:500], "...")  # Print first 500 chars

    print("[4] Summarizing Text...")
    summary = summarize_text(extracted_text)

    print("\n** Video Summary **")
    print(summary)
    return summary

# Run the pipeline with a YouTube Video
video_url = "https://www.youtube.com/watch?v=K27diMbCsuw"  # Replace with your video link
summary = video_summarization_pipeline(video_url)


[1] Downloading Video...
[youtube] Extracting URL: https://www.youtube.com/watch?v=K27diMbCsuw
[youtube] K27diMbCsuw: Downloading webpage
[youtube] K27diMbCsuw: Downloading tv client config
[youtube] K27diMbCsuw: Downloading player f6e09c70
[youtube] K27diMbCsuw: Downloading tv player API JSON
[youtube] K27diMbCsuw: Downloading ios player API JSON
[youtube] K27diMbCsuw: Downloading m3u8 information
[info] K27diMbCsuw: Downloading 1 format(s): 401+140
[download] Destination: video.f401.mp4
[download] 100% of  114.45MiB in 00:00:05 at 21.16MiB/s  
[download] Destination: video.f140.m4a
[download] 100% of    3.98MiB in 00:00:00 at 29.84MiB/s  
[Merger] Merging formats into "video.mp4"
Deleting original file video.f140.m4a (pass -k to keep)
Deleting original file video.f401.mp4 (pass -k to keep)
[2] Extracting Audio...
[3] Transcribing Audio...


100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 135MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper


Extracted Text Preview:  Hi, I'm Pete from Manus AI. For the past year, we've been quietly building what we believe is the next evolution in AI. And today, we're launching an early preview of Manus, the first general AI agent. This isn't just another chat-border workflow. It's a truly autonomous agent that bridges the gap between conception and execution. While other AI stops at generating ideas, Manus delivers results. We see it as the next paradigm of human-machine collaboration, and potentially, it glimps into AGI.  ...
[4] Summarizing Text...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



** Video Summary **
Manus is the first general AI agent that bridges the gap between conception and execution. Manus operates as a multi-agent system powered by several distinct models. The name Manus comes from the famous model, MENSE, at Manus, mind and hand.
